# Исследование надёжности заёмщиков

## Содержание: <a name="content"></a>
- [Импорт библиотек, загрузка и просмотр данных](#loading)
- [Предодработка данных](#preprocessing)
    * [Обработка пропусков](#isnull)
    * [Замена типа данных](#data_type)
    * [Обработка дубликатов](#data_type)
    * [Лемматизация](#lemmatization)
    * [Категоризация данных](#categorization)
- [Исследовательский анализ](#eda)
- [Выводы](#results)

### Импорт библиотек, загрузка и просмотр данных  <a name="loading"></a>

[к содержанию](#content)

In [2]:
# Импортируем библиотеку pandas и открываем файл
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# Смотрим общую информацию по таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# Проверяем наличие дублей, так как глазами не увидела
df.duplicated().sum()

54

### Вывод: 


В полученной таблице 12 столбцов и 21525 строк. Каждая строка содержит информацию о заемщике. В таблице есть дублирующие значения, пропуски, использование в столбцах разного регистра и разной формы слов. Также столбец `days_employed`, который должен отражать общий трудовой стаж в днях, заполнен недостоверными данными. Также необходимо заменить вещественный тип данных на целочисленный в столбце `total_income`. 

### Предобработка данных  <a name="preprocessing"></a>

### Обработка пропусков  <a name="isnull"></a>

[к содержанию](#content)

In [3]:
# Проверяем наличие пропусков в столбцах
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# Теперь видно, что пропуски есть в столбцах "total_income" и "days_employed". Начнем со столбца "total_income"
df[df['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


У клиентов банка разные типы занятости и, соответственно, доход. Нельзя заменять пропуски на общие средние или медиальные значения по всей таблице. Будем опираться на тип занятости и считать медианы отдельно для сотрудников, пенсионеров и тд 

In [5]:
# Запишем уникальные значения в переменную types
types = df['income_type'].unique()
types

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [6]:
# С помощью цикла заменим пропуски на медиану, определенную исходя из типа занятости
for i in types:
    df_median = df.loc[df['income_type'] == i, 'total_income'].median()
    df.loc[df['income_type'] == i, 'total_income'] = df.loc[df['income_type'] == i, 'total_income'].fillna(df_median)

In [7]:
# Проверим остались ли пропуски в столбце "total_income"
df[df['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [8]:
# Так же есть пропуски в столбце "days_employed". Однако, так как столбец заполнен недостоверными данными (стаж некоторых заемщиков около 1000 лет) и данный столбец не нужен для наших исследований, то можно его удалить. 
df.drop(['days_employed'], axis='columns', inplace=True)

In [9]:
# Посмотрим уникальные значения в столбце "children"
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
# Значения "-1" и "20" примем за ошибку при вводе данных и наменим на "1" и "2" соответственно
df['children'] = df['children'].replace(-1, 1)

In [11]:
df['children'] = df['children'].replace(20, 2)

In [12]:
# Проверяем изменились ли данные
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [13]:
# Проверяем остались ли еще пропуски
df.isnull().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Заменили пропуски в данных по доходу используя медиану отдельно для каждого вида занятости. Пропуски могли быть вызваны тем, что заемщики не сообщали о своем доходе. Так же может иметь место ошибка при записи данных.
Изменили некорректные значения в данных по количеству детей. Тут имеет место, скорее всего, человеческий фактор - опечатка при вводе данных клиентом или сотрудником банка. Так как значение "-1" здесь в принципе невозможно, а "20" нереалистично (в особенности для клиентов до 30 лет).
Столбец с данными по стажу удален. Так как, во-первых, был заполнен недействительными данными, во-вторых, эти данные не нужны для нашего исследования. Возможно, здесь возникает ошибка при записи данных, которую нужно проверить отдельно.

### Замена типа данных <a name="data_type"></a>

[к содержанию](#content)

In [14]:
# Заменим тип данных в столбце "total_income" с float64 на int64, используя метод astype
df['total_income'].astype('int')

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int64

### Вывод

Заменили вещественный тип данных на целочисленный методом astype.

### Обработка дубликатов  <a name="duplicates"></a>

[к содержанию](#content)

In [15]:
# Сначала изменим регистр в столбце "education" так, чтобы все значения были представленны в одном виде
df['education'].str.lower()

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21525, dtype: object

In [16]:
# Смотрим на количество дубликатов. Нас интересуют только полные совпадения
df.duplicated().sum()

54

In [17]:
# Удаляем дубликаты с перезаписью индексов
df = df.drop_duplicates().reset_index(drop = True)

In [18]:
# Проверяем остались ли дубликаты
df.duplicated().sum()

0

### Вывод

Дубликатами здесь будем считать полностью совпавшие строки. Так как нет столбца с id клиента, по которому можно было бы однозначно идентифицировать заемщика, нельзя точно говорить о том являются эти строки дубликатами или нет. Дубликаты могли появиться вследствие сбоя в записи данных

### Лемматизация  <a name="lemmatization"></a>

[к содержанию](#content)

In [19]:
# Импортируем pymystem3:
from pymystem3 import Mystem
m = Mystem()

In [20]:
# Нужно определить, какие категории целей есть в таблице
# Ищем уникальные значения в столбце "purpose", записываем их в строку, приводим слова к лемме и считаем, как часто они встречаются.
purpose_unique = df['purpose'].unique()

In [21]:
purpose_string = ' '.join(purpose_unique)

In [22]:
lemmas = m.lemmatize(purpose_string)

In [23]:
from collections import Counter
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Категории целей кредита `недвижимость`, `автомобиль`, `свадьба`, `образование`

In [24]:
# Создаем новый столбец для категорий целей
df['purpose_category'] = ''

In [25]:
# С помощью цикла проверяем к какой категории относится цель: если в строке встречается основа заданного слова, то ей назначается соответствующая категория
for i in range(len(df['purpose'])):
    if 'жиль' in df['purpose'][i]:
        df['purpose_category'][i] = 'недвижимость'
    if 'автомобил' in df['purpose'][i]:
        df['purpose_category'][i] = 'автомобиль'
    if 'недвижимост' in df['purpose'][i]:
        df['purpose_category'][i] = 'недвижимость'
    if 'свадьб' in df['purpose'][i]:
        df['purpose_category'][i] = 'свадьба'
    if 'образован' in df['purpose'][i]:
        df['purpose_category'][i] = 'образование'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_l

In [26]:
df.head(15)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость


In [27]:
# Проверим не появились ли пропуски
df.isnull().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
purpose_category    0
dtype: int64

### Вывод

Для дальнейшего исследования мы определили категории в целях кредитов. 
Выделив уникальные цели из столбца `purpose`, с помощью лемматизации были получены их категории. Всего у заемщиков оказалось четыре категориии целей кредита: `недвижимость`, `автомобиль`, `свадьба`, `образование`. Затем, с помощью цикла, для каждой цели в таблице указали ее категорию.
Можно попробовать внести изменения в форму заявки, чтобы клиентам/сотрудникам не приходилось указывать цель кредита вручную, а нужно было выбрать из предложенного списка. Тогда обработка этого столбца займет меньше времени в будущем.

### Категоризация данных  <a name="categorization"></a>

[к содержанию](#content)

Категоризируем клиентов по уровню дохода

In [28]:
# Посчитаем квантили доходов и используем их как пороговые значения для категоризации доходов
df['total_income'].describe()

count    2.147100e+04
mean     1.652959e+05
std      9.815346e+04
min      2.066726e+04
25%      1.076553e+05
50%      1.425944e+05
75%      1.957680e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [29]:
# С помощью цикла присваиваем каждой строке значение категории дохода
def income_category(income):
    if income <= 1.425944e+05:
        return 'низкий'
    if 1.425944e+05 < income <= 1.957680e+05:
        return 'средний'
    return 'высокий'

df['income_category'] = df['total_income'].apply(income_category)

In [30]:
# Добавим столбец "presence_of_children", чтобы выделить среди заемщиков отдельно бездетных и тех, у кого дети есть
def presence_of_children(children):
    if children == 0:
        return 'нет детей'
    return 'есть дети'

In [31]:
df['presence_of_children'] = df['children'].apply(presence_of_children)
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category,presence_of_children
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,высокий,есть дети
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,низкий,есть дети
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,средний,нет детей
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,высокий,есть дети
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,средний,нет детей


### Вывод

Для каждого заемщика добавили значение `уровень дохода` и `наличие детей` для того, чтобы посчитать их взаимосвять с возвратом кредита в срок

### Исследовательский анализ  <a name="eda"></a>

[к содержанию](#content)

#### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
# Сгруппируем заемщиков по наличию детей и посчитаем, сколько попала в каждую группу
presence_of_children_group = df['presence_of_children'].value_counts()
presence_of_children_group

нет детей    14107
есть дети     7364
Name: presence_of_children, dtype: int64

In [33]:
# Создадим DataFrame, где будет группированные по наличию детей заемщики
presence_of_children_group_x = [{"presence_of_children": 'нет детей', "client_sum":14107}, {"presence_of_children": 'есть дети', "client_sum":7364}]
presence_of_children_group_table = pd.DataFrame(presence_of_children_group_x)
presence_of_children_group_table

,presence_of_children,client_sum
0,нет детей,14107
1,есть дети,7364


In [34]:
# Посчитаем количество просроченных кредитов в каждой категории и объединим с предыдущей таблицей
presence_of_children_group_debt = df.pivot_table(index='presence_of_children', values='debt', aggfunc='sum')
presence_of_children_pivot = presence_of_children_group_table.merge(presence_of_children_group_debt, on='presence_of_children', how='left')
presence_of_children_pivot

,presence_of_children,client_sum,debt
0,нет детей,14107,1063
1,есть дети,7364,678


In [35]:
# Посчитаем долю просроченных кредитов в % в каждой категории и объединим с предыдущей таблицей
presence_of_children_group_debt_percent = df.pivot_table(index = 'presence_of_children', values = 'debt')
presence_of_children_group_debt_percent = presence_of_children_group_debt_percent * 100
presence_of_children_group_debt_percent = presence_of_children_group_debt_percent.rename(columns={'debt': 'доля должников в %'})
presence_of_children_pivot = presence_of_children_pivot.merge(presence_of_children_group_debt_percent, on='presence_of_children', how='left')
presence_of_children_pivot

,presence_of_children,client_sum,debt,доля должников в %
0,нет детей,14107,1063,7.535266
1,есть дети,7364,678,9.206953


In [47]:
# Посчитаем долю кредитов в % для каждой категории и объединим с предыдущей таблицей
presence_of_children_pivot['доля кредитов в %'] = presence_of_children_pivot['client_sum']/presence_of_children_pivot['client_sum'].sum()*100
presence_of_children_pivot.sort_values(by = 'доля должников в %', ascending = False)

,presence_of_children,client_sum,debt,доля должников в %,доля кредитов в %
1,есть дети,7364,678,9.206953,34.297424
0,нет детей,14107,1063,7.535266,65.702576


Также посмотрим зависимость по количеству детей.

In [36]:
# Сгруппируем заемщиков по количеству детей и посчитаем, сколько попала в каждую группу
children_group = df['children'].value_counts()

In [37]:
# Создадим DataFrame, где будет группированные по количеству детей заемщики
children_group_x = [{"children": 0, "client_sum":14107}, {"children": 1, "client_sum":4856}, {"children": 2, "client_sum":2128}, {"children": 3, "client_sum":330}, {"children": 4, "client_sum":41}, {"children": 5, "client_sum":9}]
children_group_table = pd.DataFrame(children_group_x)
children_group_table

,children,client_sum
0,0,14107
1,1,4856
2,2,2128
3,3,330
4,4,41
5,5,9


In [38]:
# Посчитаем количество просроченных кредитов в каждой категории и объединим с предыдущей таблицей
children_group_debt = df.pivot_table(index='children', values='debt', aggfunc='sum')
children_pivot = children_group_table.merge(children_group_debt, on='children', how='left')
children_pivot

,children,client_sum,debt
0,0,14107,1063
1,1,4856,445
2,2,2128,202
3,3,330,27
4,4,41,4
5,5,9,0


In [39]:
# Посчитаем долю просроченных кредитов в % в каждой категории и объединим с предыдущей таблицей
children_group_debt_percent = df.pivot_table(index = 'children', values = 'debt')
children_group_debt_percent = children_group_debt_percent * 100
children_group_debt_percent = children_group_debt_percent.rename(columns={'debt': 'доля должников в %'})
children_pivot = children_pivot.merge(children_group_debt_percent, on='children', how='left')

In [40]:
children_pivot

,children,client_sum,debt,доля должников в %
0,0,14107,1063,7.535266
1,1,4856,445,9.163921
2,2,2128,202,9.492481
3,3,330,27,8.181818
4,4,41,4,9.756098
5,5,9,0,0.000000


In [48]:
# Посчитаем долю кредитов в % для каждой категории и объединим с предыдущей таблицей
children_pivot['доля кредитов по кол-ву детей в %'] = children_pivot['client_sum']/children_pivot['client_sum'].sum()*100
children_pivot.sort_values(by = 'доля должников в %', ascending = False)

,children,client_sum,debt,доля должников в %,доля кредитов по кол-ву детей в %
4,4,41,4,9.756098,0.190955
2,2,2128,202,9.492481,9.911043
1,1,4856,445,9.163921,22.616553
3,3,330,27,8.181818,1.536957
0,0,14107,1063,7.535266,65.702576
5,5,9,0,0.000000,0.041917


### Вывод

Клиенты, у котрых нет детей, чаще отдают кредиты в срок (доля задолжников у них 7,5 % против 9,2%). 

#### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [42]:
# Сгруппируем заемщиков по семейному положению и посчитаем, сколько попала в каждую группу
family_status_group = df['family_status'].value_counts()
family_status_group

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [43]:
# Создадим DataFrame, где будет группированные заемщики по семейному положению 
family_status_group_x = [{"family_status": 'женат / замужем', "client_sum":12344}, {"family_status": 'гражданский брак', "client_sum":4163}, {"family_status": 'Не женат / не замужем', "client_sum":2810}, {"family_status": 'в разводе', "client_sum":1195}, {"family_status": 'вдовец / вдова', "client_sum":959}]
family_status_group_table = pd.DataFrame(family_status_group_x)
family_status_group_table

,family_status,client_sum
0,женат / замужем,12344
1,гражданский брак,4163
2,Не женат / не замужем,2810
3,в разводе,1195
4,вдовец / вдова,959


In [44]:
# Посчитаем количество просроченных кредитов в каждой категории и объединим с предыдущей таблицей
family_status_group_debt = df.pivot_table(index='family_status', values='debt', aggfunc='sum')
family_status_pivot = family_status_group_table.merge(family_status_group_debt, on='family_status', how='left')
family_status_pivot

,family_status,client_sum,debt
0,женат / замужем,12344,931
1,гражданский брак,4163,388
2,Не женат / не замужем,2810,274
3,в разводе,1195,85
4,вдовец / вдова,959,63


In [45]:
# Посчитаем долю просроченных кредитов в % в каждой категории и объединим с предыдущей таблицей
family_status_group_debt_percent = df.pivot_table(index = 'family_status', values = 'debt')
family_status_group_debt_percent = family_status_group_debt_percent * 100
family_status_group_debt_percent = family_status_group_debt_percent.rename(columns={'debt': 'доля должников в %'})
family_status_pivot = family_status_pivot.merge(family_status_group_debt_percent, on='family_status', how='left')
family_status_pivot

,family_status,client_sum,debt,доля должников в %
0,женат / замужем,12344,931,7.542126
1,гражданский брак,4163,388,9.320202
2,Не женат / не замужем,2810,274,9.750890
3,в разводе,1195,85,7.112971
4,вдовец / вдова,959,63,6.569343


In [49]:
# Посчитаем долю кредитов в % для каждой категории и объединим с предыдущей таблицей
family_status_pivot['доля кредитов по семейному положению в %'] = family_status_pivot['client_sum']/family_status_pivot['client_sum'].sum()*100
family_status_pivot.sort_values(by = 'доля должников в %', ascending = False)

,family_status,client_sum,debt,доля должников в %,доля кредитов по семейному положению в %
2,Не женат / не замужем,2810,274,9.750890,13.087420
1,гражданский брак,4163,388,9.320202,19.388943
0,женат / замужем,12344,931,7.542126,57.491500
3,в разводе,1195,85,7.112971,5.565647
4,вдовец / вдова,959,63,6.569343,4.466490


### Вывод

Наименее надежными оказались неженатые/незамужние или проживающие в гразданском браке клиенты. Здесь 9,6% и 9,3% должников, соответсвенно, для остальных групп этот показатель менее/равен 7,5%

#### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [50]:
# Сгруппируем заемщиков по уровню дохода и посчитаем, сколько попала в каждую группу
income_category_group = df['income_category'].value_counts()
income_category_group


низкий     10858
высокий     5368
средний     5245
Name: income_category, dtype: int64

In [51]:
# Создадим DataFrame, где будет группированные по уровню дохода заемщики
income_category_group_x = [{"income_category": 'низкий', "client_sum":10858}, {"income_category": 'средний', "client_sum":5245},  {"income_category": 'высокий', "client_sum":5368}]
income_category_group_table = pd.DataFrame(income_category_group_x)
income_category_group_table

,income_category,client_sum
0,низкий,10858
1,средний,5245
2,высокий,5368


In [52]:
# Посчитаем количество просроченных кредитов в каждой категории и объединим с предыдущей таблицей
income_category_group_debt = df.pivot_table(index='income_category', values='debt', aggfunc='sum')
income_category_pivot = income_category_group_table.merge(income_category_group_debt, on='income_category', how='left')
income_category_pivot

,income_category,client_sum,debt
0,низкий,10858,910
1,средний,5245,448
2,высокий,5368,383


In [53]:
# Посчитаем долю просроченных кредитов в % в каждой категории и объединим с предыдущей таблицей
income_category_group_debt_percent = df.pivot_table(index = 'income_category', values = 'debt')
income_category_group_debt_percent = income_category_group_debt_percent * 100
income_category_group_debt_percent = income_category_group_debt_percent.rename(columns={'debt': 'доля должников в %'})
income_category_pivot = income_category_pivot.merge(income_category_group_debt_percent, on='income_category', how='left')
income_category_pivot

,income_category,client_sum,debt,доля должников в %
0,низкий,10858,910,8.380917
1,средний,5245,448,8.541468
2,высокий,5368,383,7.134873


In [54]:
# Посчитаем долю кредитов в % для каждой категории и объединим с предыдущей таблицей
income_category_pivot['доля кредитов по наличию детей в %'] = income_category_pivot['client_sum']/income_category_pivot['client_sum'].sum()*100
income_category_pivot.sort_values(by = 'доля должников в %', ascending = False)

,income_category,client_sum,debt,доля должников в %,доля кредитов по наличию детей в %
1,средний,5245,448,8.541468,24.428299
0,низкий,10858,910,8.380917,50.570537
2,высокий,5368,383,7.134873,25.001164


### Вывод

Клиенты со средним доходом оказались наименее надежными (8,5% должников). Большая часть кредитов (50,6%) была выдана тем, чей доход ниже среднего. Доля должников в этой категории близка к клиентам со средним доходом (8,4%)

#### - Как разные цели кредита влияют на его возврат в срок?

In [55]:
# Сгруппируем заемщиков по категории целей и посчитаем, сколько попала в каждую группу
purpose_category_group = df['purpose_category'].value_counts()
purpose_category_group

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose_category, dtype: int64

In [56]:
# Создадим DataFrame, где будет группированные по категории целей заемщики
purpose_category_group_x = [{"purpose_category": 'недвижимость', "client_sum":10814}, {"purpose_category": 'автомобиль', "client_sum":4308}, {"purpose_category": 'образование', "client_sum":4014}, {"purpose_category": 'свадьба', "client_sum":2335}]
purpose_category_group_table = pd.DataFrame(purpose_category_group_x)
purpose_category_group_table


,purpose_category,client_sum
0,недвижимость,10814
1,автомобиль,4308
2,образование,4014
3,свадьба,2335


In [57]:
# Посчитаем количество просроченных кредитов в каждой категории и объединим с предыдущей таблицей
purpose_category_group_debt = df.pivot_table(index='purpose_category', values='debt', aggfunc='sum')
purpose_category_pivot = purpose_category_group_table.merge(purpose_category_group_debt, on='purpose_category', how='left')
purpose_category_pivot

,purpose_category,client_sum,debt
0,недвижимость,10814,782
1,автомобиль,4308,403
2,образование,4014,370
3,свадьба,2335,186


In [58]:
# Посчитаем долю просроченных кредитов в % в каждой категории и объединим с предыдущей таблицей
purpose_category_group_debt_percent = df.pivot_table(index = 'purpose_category', values = 'debt')
purpose_category_group_debt_percent = purpose_category_group_debt_percent * 100
purpose_category_group_debt_percent = purpose_category_group_debt_percent.rename(columns={'debt': 'доля должников в %'})
purpose_category_pivot = purpose_category_pivot.merge(purpose_category_group_debt_percent, on='purpose_category', how='left')
purpose_category_pivot

,purpose_category,client_sum,debt,доля должников в %
0,недвижимость,10814,782,7.231367
1,автомобиль,4308,403,9.354689
2,образование,4014,370,9.217738
3,свадьба,2335,186,7.965739


In [59]:
# Посчитаем долю кредитов в % для каждой категории и объединим с предыдущей таблицей
purpose_category_pivot['доля кредитов по наличию детей в %'] = purpose_category_pivot['client_sum']/purpose_category_pivot['client_sum'].sum()*100
purpose_category_pivot.sort_values(by = 'доля должников в %', ascending = False)

,purpose_category,client_sum,debt,доля должников в %,доля кредитов по наличию детей в %
1,автомобиль,4308,403,9.354689,20.064273
2,образование,4014,370,9.217738,18.694984
3,свадьба,2335,186,7.965739,10.875134
0,недвижимость,10814,782,7.231367,50.365609


### Вывод

Больше всего кредитов берут для операций с недвижимостью. При этом клиенты, взявшие кредит под данные цели, выплачивают его в срок чаще других (7,2% должников). Хуже всего оплачиваются кредиты в категории цели "автомобиль" (9,4 % должников).

### Общий вывод  <a name="results"></a>

[к содержанию](#content)

На основании полученных данных можно сказать, что каждый из изученных фактов влияет на возврат кредитов в срок.
При отдельном рассмотрении по каждому пункту, наиболее надежными являются клиенты, у которых:
- нет делей
- состоят/состояли в браке 
- имеют высокий доход
- берут кредит для операций с недвижимостью.
Однако, при определении надежности стоит смотреть не на отдельные пункты, а учитывая полную картину.
Стоит обратить внимание на тот, как собираются данные по трудовому стажу, так как одни выглядят недостоверными. А так же уделить внимание причинам появления пропусков в столбцах со стажем и доходом.
Можно внести изменения в форму заявки, чтобы при ее заполнении для пунктов "цели" и "уровень образования" не нужно было указавать информацию вручную. Выбор из предложенного списка сократит время на обработку данных, так как не будет необходимости менять регистр и вычленять категории целей.